In [1]:
import numpy as np
from rips import *
from rips.utils import FeynmanKac, Particle

# 1. Setting up model

Here we consider the stochastic system $g(X) = \beta  - \frac{1}{\sqrt{n}} \sum_{i=1}^{n} X_i$, where $(X_i)$ are i.i.d. standard normal random variables.

We are interested in the probability $p = P\{g(X)\leq 0\}$.

Note that when $\beta \to \infty$ we have that $p \to 0$, so crude Monte Carlo samling becomes unfeasible for large $\beta$. 

Next we show how to use pims to approximate the probability $p$.

In [2]:
# Define model with linear limit state function
class LinearLSF(FeynmanKac, StandardGaussian):
    def __init__(self, beta=1, n=10, **kwargs):
        self.beta = beta
        self.n = n
        super().__init__(**kwargs)
    
    def score_function(self, particle: Particle) -> float:
        particle.response = particle.path.sum() / np.sqrt(self.n)
        return particle.response / self.beta
    
    @property
    def num_variables(self) -> int:
        return self.n

model = LinearLSF(beta=6, n=1000)

In [3]:
# Sample a particle
particle = model.sample_particle()

In [4]:
# This is the score function evaluated on a random vector
particle.score  # Greater than one signifies event of interest g(X) <= 0

-0.13792925318289992

# 2. Probabilistic inference

In [5]:
# Number of samples
model.num_of_particles = 10000
# Markov Chain Monte Carlo methods
kernels = [
    PCN(),
    # MMH()
]
# Sample-based Monte Carlo integration
sample = ComboUQ(model, kernels)
sample.summary_results()


Adaptive levels...
[-inf, 0.13891543474600263]
[-inf, 0.13891543474600263, 0.2856820035665473]
[-inf, 0.13891543474600263, 0.2856820035665473, 0.4010426081789563]
[-inf, 0.13891543474600263, 0.2856820035665473, 0.4010426081789563, 0.49081591352523235]
[-inf, 0.13891543474600263, 0.2856820035665473, 0.4010426081789563, 0.49081591352523235, 0.569599038921804]
[-inf, 0.13891543474600263, 0.2856820035665473, 0.4010426081789563, 0.49081591352523235, 0.569599038921804, 0.6408751412681312]
[-inf, 0.13891543474600263, 0.2856820035665473, 0.4010426081789563, 0.49081591352523235, 0.569599038921804, 0.6408751412681312, 0.7008175938164891]
[-inf, 0.13891543474600263, 0.2856820035665473, 0.4010426081789563, 0.49081591352523235, 0.569599038921804, 0.6408751412681312, 0.7008175938164891, 0.7571157746221547]
[-inf, 0.13891543474600263, 0.2856820035665473, 0.4010426081789563, 0.49081591352523235, 0.569599038921804, 0.6408751412681312, 0.7008175938164891, 0.7571157746221547, 0.8115499537757305]
[-inf, 0

{'cpu_time_smc': 12.560560375,
 'p_smc': 8.0592896e-10,
 'ng_smc': 140001,
 'cpu_time_gs': 12.469817374999998,
 'p_bar': 9.387212800000002e-10,
 'ng_gs': 146950,
 'k': 14,
 'var': 1.8415964470528175e-16,
 'std': 1.3570543272296867e-08}

# 3. Compare approximations to exact solution

In [6]:
from scipy.stats import norm
p_exact = norm.cdf(-6)
p_exact

9.865876450376946e-10

In [7]:
# Unbiased estimator
sample.p_bar

9.387212800000002e-10

In [8]:
# relative error
abs(sample.p_bar - p_exact) / p_exact

0.04851709351769308

In [9]:
# Biased estimator
sample.p_smc

8.0592896e-10

In [10]:
# relative error
abs(sample.p_smc - p_exact) / p_exact

0.18311468418073704